In [1]:
pip install numpy pandas opencv-python tensorflow scikit-learn

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.26.4 which is incompatible.


In [2]:
import pandas as pd #importing #importing
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from glob import glob
import random
import matplotlib.pyplot as plt

In [3]:
def load_fer2013(csv_file): #loading the datasets
    data = pd.read_csv(csv_file)
    pixels = data['pixels'].tolist()
    images = np.array([np.fromstring(pixel_sequence, sep=' ') for pixel_sequence in pixels], dtype='float32')
    images = images.reshape((images.shape[0], 48, 48, 1))
    labels = pd.get_dummies(data['emotion']).values
    return images, labels

In [5]:
def preprocess_images(images):
    images = images / 255.0
    return images

In [6]:
dataset_path = 'C://Users//Shweta//OneDrive//Desktop//fer2013.csv'
 

In [7]:
if not os.path.exists(dataset_path):
    print(f"Dataset not found at {dataset_path}")
else:
    data = pd.read_csv(dataset_path)

In [9]:
images, labels = load_fer2013(dataset_path)
images = preprocess_images(images)
print(images.shape, labels.shape)

(35887, 48, 48, 1) (35887, 7)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42) #splitting the dataset

In [ ]:
print(f'Training data shape: {X_train.shape}')
print(f'Testing data shape: {X_test.shape}')
print(f'Training labels shape: {y_train.shape}')
print(f'Testing labels shape: {y_test.shape}') #printing the shapes of the datasets

Training data shape: (28709, 48, 48, 1)
Testing data shape: (7178, 48, 48, 1)
Training labels shape: (28709, 7)
Testing labels shape: (7178, 7)


In [ ]:
# Splitting dataset into different contexts
def split_by_context(images, labels):
    context_labels = []
   
    half_size = len(images) // 2
    for i in range(len(images)):
        if i < half_size:
            context_labels.append('context1')  # e.g., bright lighting
        else:
            context_labels.append('context2')  # e.g., dim lighting
    
    context_labels = np.array(context_labels)
    return context_labels

context_labels = split_by_context(images, labels)
print(context_labels.shape)

(35887,)


In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_model(input_shape):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train separate models for different contexts (assuming we have context labels)
# Here, we'll just train one model as an example
input_shape = (48, 48, 1)
model = create_model(input_shape)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))



Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 62s 68ms/step - accuracy: 0.2592 - loss: 1.8027 - val_accuracy: 0.3867 - val_loss: 1.5907
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 38s 42ms/step - accuracy: 0.3724 - loss: 1.5991 - val_accuracy: 0.4347 - val_loss: 1.4817
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 38s 39ms/step - accuracy: 0.4188 - loss: 1.5069 - val_accuracy: 0.4678 - val_loss: 1.4080
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.4439 - loss: 1.4416 - val_accuracy: 0.4822 - val_loss: 1.3742
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 34s 38ms/step - accuracy: 0.4561 - loss: 1.4138 - val_accuracy: 0.4953 - val_loss: 1.3410
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 313s 348ms/step - accuracy: 0.4760 - loss: 1.3686 - val_accuracy: 0.4916 - val_loss: 1.3310
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 60s 65ms/step - accuracy: 0.4783 - loss: 1.3554 - val_accuracy: 0.5085 - val_loss: 1.2983
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 66s 73ms/step - accuracy: 0.4885 - loss: 1.3324 

In [23]:
from tensorflow.keras.layers import Input, Concatenate

# Example context data (e.g., location, cultural background)
# For simplicity, we'll generate random context data
context_train = np.random.rand(X_train.shape[0], 5)
context_test = np.random.rand(X_test.shape[0], 5)

def create_multi_context_model(input_shape, context_dim):
    image_input = Input(shape=input_shape)
    context_input = Input(shape=(context_dim,))

    x = Conv2D(32, kernel_size=(3, 3), activation='relu')(image_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)

    x = Concatenate()([x, context_input])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(7, activation='softmax')(x)

    model = tf.keras.Model(inputs=[image_input, context_input], outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

context_dim = 5
multi_context_model = create_multi_context_model((48, 48, 1), context_dim)
multi_context_model.fit([X_train, context_train], y_train, epochs=10, validation_data=([X_test, context_test], y_test))



Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 40s 41ms/step - accuracy: 0.2553 - loss: 1.7985 - val_accuracy: 0.3781 - val_loss: 1.6158
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 38s 43ms/step - accuracy: 0.3681 - loss: 1.6103 - val_accuracy: 0.4210 - val_loss: 1.5033
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 39s 43ms/step - accuracy: 0.4138 - loss: 1.5197 - val_accuracy: 0.4416 - val_loss: 1.4430
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 37s 41ms/step - accuracy: 0.4413 - loss: 1.4637 - val_accuracy: 0.4668 - val_loss: 1.3931
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 60s 67ms/step - accuracy: 0.4436 - loss: 1.4339 - val_accuracy: 0.4804 - val_loss: 1.3627
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 79s 63ms/step - accuracy: 0.4628 - loss: 1.3981 - val_accuracy: 0.4911 - val_loss: 1.3404
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 50s 56ms/step - accuracy: 0.4745 - loss: 1.3675 - val_accuracy: 0.4983 - val_loss: 1.3127
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 48s 53ms/step - accuracy: 0.4834 - loss: 1.3451 - 

In [27]:
loss, accuracy = multi_context_model.evaluate([X_test, context_test], y_test) #generalizing the datsets
print(f"Test accuracy: {accuracy * 100:.2f}%")

225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5239 - loss: 1.2767
Test accuracy: 52.24%
